In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
!ls /home/f/F.Kiessler/makeathon/data_india/

agriculture_commodity_apmc_msp	crop_recommendation_dataset
crop_production_in_india	data_census_india_centroids


# Show all Datasets

In [3]:
#agriculture_commodity_apmc_msp
cmo_msp_mandi = pd.read_csv("/home/f/F.Kiessler/makeathon/data_india/agriculture_commodity_apmc_msp/CMO_MSP_Mandi.csv")
monthly_data_cmo = pd.read_csv("/home/f/F.Kiessler/makeathon/data_india/agriculture_commodity_apmc_msp/Monthly_data_cmo.csv")

#crop_recommendation_dataset
crop_recommendation = pd.read_csv("/home/f/F.Kiessler/makeathon/data_india/crop_recommendation_dataset/Crop_recommendation.csv")

#crop_production_in_india
crop_production = pd.read_csv("/home/f/F.Kiessler/makeathon/data_india/crop_production_in_india/crop_production.csv")

In [4]:
#print(cmo_msp_mandi["commodity"].unique())
cmo_msp_mandi.head()

,commodity,year,Type,msprice,msp_filter
0,PADDY-UNHUSKED,2012,Kharif Crops,1250.0,1
1,RICE(PADDY-HUS),2012,Kharif Crops,1280.0,1
2,Jowar_Hybrid,2012,Kharif Crops,1500.0,1
3,SORGUM(JAWAR),2012,Kharif Crops,1520.0,1
4,BAJRI,2012,Kharif Crops,1175.0,1


In [5]:
#print(monthly_data_cmo["Commodity"].unique())
monthly_data_cmo.head()

,APMC,Commodity,Year,Month,arrivals_in_qtl,min_price,max_price,modal_price,date,district_name,state_name
0,Ahmednagar,Bajri,2015,April,79,1406,1538,1463,2015-04,Ahmadnagar,Maharashtra
1,Ahmednagar,Bajri,2016,April,106,1788,1925,1875,2016-04,Ahmadnagar,Maharashtra
2,Ahmednagar,Wheat(Husked),2015,April,1253,1572,1890,1731,2015-04,Ahmadnagar,Maharashtra
3,Ahmednagar,Wheat(Husked),2016,April,387,1750,2220,1999,2016-04,Ahmadnagar,Maharashtra
4,Ahmednagar,Sorgum(Jawar),2015,April,3825,1600,2200,1900,2015-04,Ahmadnagar,Maharashtra


In [6]:
print(crop_recommendation["label"].unique())
crop_recommendation.head()

['rice' 'maize' 'chickpea' 'kidneybeans' 'pigeonpeas' 'mothbeans'
 'mungbean' 'blackgram' 'lentil' 'pomegranate' 'banana' 'mango' 'grapes'
 'watermelon' 'muskmelon' 'apple' 'orange' 'papaya' 'coconut' 'cotton'
 'jute' 'coffee']


,N,P,K,temperature,humidity,ph,rainfall,label
0,90,42,43,20.879744,82.002744,6.502985,202.935536,rice
1,85,58,41,21.770462,80.319644,7.038096,226.655537,rice
2,60,55,44,23.004459,82.320763,7.840207,263.964248,rice
3,74,35,40,26.491096,80.158363,6.980401,242.864034,rice
4,78,42,42,20.130175,81.604873,7.628473,262.717340,rice


In [7]:
crop_production.head()

,State_Name,District_Name,Crop_Year,Season,Crop,Area,Production
0,Andaman and Nicobar Islands,NICOBARS,2000,Kharif,Arecanut,1254.0,2000.0
1,Andaman and Nicobar Islands,NICOBARS,2000,Kharif,Other Kharif pulses,2.0,1.0
2,Andaman and Nicobar Islands,NICOBARS,2000,Kharif,Rice,102.0,321.0
3,Andaman and Nicobar Islands,NICOBARS,2000,Whole Year,Banana,176.0,641.0
4,Andaman and Nicobar Islands,NICOBARS,2000,Whole Year,Cashewnut,720.0,165.0


# Dataset4: crop effectivenes in Location:

In [8]:
df = crop_production[['District_Name', 'Crop', 'Area', 'Production']].copy()

f = {
        'District_Name' : 'nunique', 
        'Area' : 'sum',
        'Production' : 'sum'
}

g = df.groupby(['District_Name',"Crop"])
v1 = g.agg(f)
v2 = g.agg(lambda x: x.drop_duplicates('Crop', keep='first').Crop.sum())

df = pd.concat([v1, v2.to_frame('Crop')], 1)

df = df.drop(["Crop","District_Name"], axis=1)
df = df.reset_index(level=['Crop'])
df["efficacy"] = df["Production"]/df["Area"]


In [9]:
df

,Crop,Area,Production,efficacy
District_Name,,,,
24 PARAGANAS NORTH,Arecanut,16591.0,38328.0,2.310168
24 PARAGANAS NORTH,Arhar/Tur,348.0,335.0,0.962644
24 PARAGANAS NORTH,Coconut,37643.0,122664239.8,3258.620190
24 PARAGANAS NORTH,Cotton(lint),4479.0,4786.0,1.068542
24 PARAGANAS NORTH,Dry chillies,46595.0,81182.0,1.742290
...,...,...,...,...
ZUNHEBOTO,Tapioca,1080.0,21440.0,19.851852
ZUNHEBOTO,Tea,395.0,835.0,2.113924
ZUNHEBOTO,Turmeric,50.0,200.0,4.000000


In [10]:
# dist vs best crop
labelled_dist_crop = {}
for dist in df.index.unique():
    argmax = df.loc[dist]["efficacy"].argmax()
    if len(df.loc[dist].shape) > 1:
        crop = df.loc[dist].iloc[int(argmax)]["Crop"]
        labelled_dist_crop[dist] = crop
    else: #if only one entry, suggest that entry
        crop = df.loc[dist]["Crop"]
        labelled_dist_crop[dist] = crop


In [51]:
end_df = pd.DataFrame([labelled_dist_crop])

end_df = end_df.transpose()
end_df = end_df.reset_index()
end_df.rename(columns = {'index':'dist', 0:"crop"}, inplace = True)
end_df.to_csv("districts_vs_crops.csv")
end_df

,dist,crop
0,24 PARAGANAS NORTH,Coconut
1,24 PARAGANAS SOUTH,Coconut
2,ADILABAD,Sugarcane
3,AGAR MALWA,Sugarcane
4,AGRA,Sugarcane
...,...,...
641,YADGIR,Sugarcane
642,YAMUNANAGAR,Sugarcane
643,YANAM,Coconut
644,YAVATMAL,Sugarcane


In [12]:
# dist, best crop, eff
labelled_dist_crop_eff = []
for dist in df.index.unique():
    argmax = df.loc[dist]["efficacy"].argmax()
    if len(df.loc[dist].shape) > 1:
        crop = df.loc[dist].iloc[int(argmax)]["Crop"]
        effi = df.loc[dist].iloc[int(argmax)]["efficacy"]
        labelled_dist_crop_eff.append({"dist":dist, "crop":crop, "eff":effi})
    else: #if only one entry, suggest that entry
        crop = df.loc[dist]["Crop"]
        effi = df.loc[dist]["efficacy"]
        labelled_dist_crop_eff.append({"dist":dist, "crop":crop, "eff":effi})

In [13]:
end_df_eff = pd.DataFrame.from_dict(labelled_dist_crop_eff)
end_df_eff.to_csv("districts_vs_crops_with_efficacy.csv")
no_best_crops = len(set(end_df_eff["crop"]))
end_df_eff.head()

,dist,crop,eff
0,24 PARAGANAS NORTH,Coconut,3258.620190
1,24 PARAGANAS SOUTH,Coconut,2244.891828
2,ADILABAD,Sugarcane,82.385214
3,AGAR MALWA,Sugarcane,37.857143
4,AGRA,Sugarcane,48.458864


In [14]:
end_df_eff.iloc[633]

dist    WEST GODAVARI
crop         Coconut 
eff      14905.128931
Name: 633, dtype: object

In [15]:
set(end_df_eff["crop"])

{'Banana',
 'Barley',
 'Coconut ',
 'Dry chillies',
 'Dry ginger',
 'Maize',
 'Onion',
 'Other Vegetables',
 'Papaya',
 'Pineapple',
 'Potato',
 'Rice',
 'Sugarcane',
 'Sweet potato',
 'Tapioca',
 'Tobacco'}

# read in the price for each crop in 2016

In [16]:
price_df = cmo_msp_mandi
price_df["commodity"] = price_df["commodity"].str.upper()
price_df["msprice"] = price_df["msprice"].fillna(0)

crop_price_dict = []
for crop in list(set(end_df_eff["crop"])):
    if len(price_df[ (price_df["commodity"]==crop.upper()) ]) > 0: #if this series isnt empty
        crop_price_dict.append({crop: np.max(price_df[ (price_df["commodity"]==crop.upper()) ]["msprice"])})
print(crop_price_dict)
no_d1_crops = len(crop_price_dict)

[{'Sugarcane': 230.0}, {'Maize': 1365.0}]


In [17]:
price_df = monthly_data_cmo
price_df["Commodity"] = price_df["Commodity"].str.upper()
crop_price_dict = []
for crop in list(set(end_df_eff["crop"])):
    if len(price_df[ (price_df["Commodity"]==crop.upper()) ]) > 0: #if this series isnt empty
        crop_price_dict.append({crop:np.mean( price_df[ (price_df["Commodity"]==crop.upper() )]["modal_price"] )})
print(crop_price_dict)
no_d2_crops = len(crop_price_dict)

[{'Pineapple': 1578.1150442477876}, {'Banana': 886.0364741641338}, {'Sugarcane': 696.0}, {'Maize': 1355.8165750196385}, {'Onion': 1283.4839743589744}, {'Sweet potato': 1549.0184049079755}, {'Other Vegetables': 1306.3866666666668}, {'Potato': 1262.8955431754875}]


In [18]:
print(no_best_crops, no_d1_crops, no_d2_crops)
# but i know when printing theyre different

16 2 8


In [19]:
monthly_data_cmo

,APMC,Commodity,Year,Month,arrivals_in_qtl,min_price,max_price,modal_price,date,district_name,state_name
0,Ahmednagar,BAJRI,2015,April,79,1406,1538,1463,2015-04,Ahmadnagar,Maharashtra
1,Ahmednagar,BAJRI,2016,April,106,1788,1925,1875,2016-04,Ahmadnagar,Maharashtra
2,Ahmednagar,WHEAT(HUSKED),2015,April,1253,1572,1890,1731,2015-04,Ahmadnagar,Maharashtra
3,Ahmednagar,WHEAT(HUSKED),2016,April,387,1750,2220,1999,2016-04,Ahmadnagar,Maharashtra
4,Ahmednagar,SORGUM(JAWAR),2015,April,3825,1600,2200,1900,2015-04,Ahmadnagar,Maharashtra
...,...,...,...,...,...,...,...,...,...,...,...
62424,Shrigonda,GRAM,2016,November,586,5700,6367,6200,2016-11,Ahmadnagar,Maharashtra
62425,Shrigonda,GREEN GRAM,2016,November,2,5000,5000,5000,2016-11,Ahmadnagar,Maharashtra
62426,Shrigonda,BLACK GRAM,2016,November,46,4700,6933,6400,2016-11,Ahmadnagar,Maharashtra
62427,Shrigonda,SOYBEAN,2016,November,166,2583,2708,2633,2016-11,Ahmadnagar,Maharashtra


# Small calculations to get turnover in Rupee for watermelons and Muskmelons

In [172]:
df.head()
monthly_data_cmo["Commodity"].unique()
cmo_msp_mandi["commodity"].unique()

df[(df["Crop"]=="Water Melon")] # the databank has some strange entry formats for some crops 
# zero production visible
# Musk Melons are not to be found in this dataset too...
# so now experimental physics inuition:

,Crop,Area,Production,efficacy
District_Name,,,,
ALWAR,Water Melon,116.0,0.0,0.0
BALAGHAT,Water Melon,2.0,0.0,0.0
BARAN,Water Melon,44.0,0.0,0.0
BARWANI,Water Melon,2.0,0.0,0.0
BHARATPUR,Water Melon,4.0,0.0,0.0
...,...,...,...,...
UJJAIN,Water Melon,3.0,0.0,0.0
VELLORE,Water Melon,23.0,0.0,0.0
VIDISHA,Water Melon,10.0,0.0,0.0


In [173]:
watermelon_per_hec = 100*100/1 # one per m^2 is 1e4 per hectar times 
muskmelon_per_hec = 1/0.7 * 1e4 # one muskmelon needs 0.7m^2 to grow, approximately 
area = 100 # in hec

In [174]:
monthly_data_cmo[ (monthly_data_cmo["Commodity"]=="MELON") ]

,APMC,Commodity,Year,Month,arrivals_in_qtl,min_price,max_price,modal_price,date,district_name,state_name
138,Rahata,MELON,2015,April,3,400,400,400,2015-04,Ahmadnagar,Maharashtra
192,Rahuri,MELON,2016,April,1301,244,1013,631,2016-04,Ahmadnagar,Maharashtra
484,Rahata,MELON,2016,August,37,326,376,326,2016-08,Ahmadnagar,Maharashtra
529,Rahuri,MELON,2016,August,25,100,233,167,2016-08,Ahmadnagar,Maharashtra
1904,Rahata,MELON,2016,July,3,500,500,500,2016-07,Ahmadnagar,Maharashtra
...,...,...,...,...,...,...,...,...,...,...,...
36417,Nashik,MELON,2016,October,20,400,1500,1000,2016-10,Nasik,Maharashtra
36786,Nashik,MELON,2016,September,220,333,833,633,2016-09,Nasik,Maharashtra
48231,Karad,MELON,2016,August,54,2000,2500,2500,2016-08,Satara,Maharashtra
59767,Nashik,MELON,2016,November,250,950,1983,1400,2016-11,Nasik,Maharashtra


In [175]:
price_per_melon = monthly_data_cmo[ (monthly_data_cmo["Commodity"]=="MELON") & (monthly_data_cmo["Year"]==2016) ]["modal_price"].mean()
price_per_melon

1266.7692307692307

In [176]:
turnover_watermelon = price_per_melon * watermelon_per_hec
turnover_muskmelon = price_per_melon * 0.1* muskmelon_per_hec # from indian supermarket sites we found the average 0.1 factor for muskmelon vs watermelon
print(turnover_watermelon)
print(turnover_muskmelon)

12667692.307692308
1809670.3296703298


# Centroid Calculation from merging of two dataframes

In [20]:
centroid_dist = pd.read_csv("/home/f/F.Kiessler/makeathon/data_india/data_census_india_centroids/district wise centroids.csv")
dlist =list(centroid_dist["District"])
doubles = set([x for x in dlist if dlist.count(x) > 1])
print("doubles: {}".format(doubles))
centroid_dist = centroid_dist.drop_duplicates(subset = "District", keep="first").set_index(["District"])#just erased the duplicates, imprecise
dictCentroidsDist = centroid_dist.to_dict("index")

doubles: {'Raigarh', 'Junagadh', 'Hamirpur', 'Bilaspur', 'Aurangabad'}


In [21]:
centroid_dist.head()

,State,Latitude,Longitude
District,,,
Andaman Islands,Andaman and Nicobar,12.382571,92.822911
Nicobar Islands,Andaman and Nicobar,7.835291,93.511601
Adilabad,Andhra Pradesh,19.284514,78.813212
Anantapur,Andhra Pradesh,14.312066,77.460158
Chittoor,Andhra Pradesh,13.331093,78.927639


In [22]:
endDfComp = end_df.set_index("dist")
endDfKeys = endDfComp.to_dict("index").keys()
centroidKeys = dictCentroidsDist.keys()

In [23]:
print(len(centroidKeys))
print(len(endDfKeys))

589
646


In [24]:
###########################
match_centroidtoendDf = {}
###########################
centroid_keys_not_in_endDfKeys = []
for centroid_key in centroidKeys:
    if centroid_key.upper() in endDfKeys:
        match_centroidtoendDf[centroid_key] = centroid_key.upper()
        continue
    else:
        centroid_keys_not_in_endDfKeys.append(centroid_key)
print("number of centroid keys not in endDfKeys: {}".format(len(centroid_keys_not_in_endDfKeys)))

endDfKeys_not_in_centroidKeys = []
upper_centroidKeys = [key.upper() for key in centroidKeys]
for endDfKey in endDfKeys:
    if endDfKey.upper() in upper_centroidKeys:

        continue
    else:
        endDfKeys_not_in_centroidKeys.append(endDfKey)
print("number of endDfKeys not in centroid keys: {}".format(len(endDfKeys_not_in_centroidKeys)))

number of centroid keys not in endDfKeys: 95
number of endDfKeys not in centroid keys: 152


Preliminary mapping is match_centroidtoendDf

In [25]:
print(centroid_keys_not_in_endDfKeys)
print(endDfKeys_not_in_centroidKeys)

['Andaman Islands', 'Nicobar Islands', 'Cuddapah', 'Nellore', 'Vishakhapatnam', 'Upper Dibang Valley', 'Dhuburi', 'North Cachar Hills', 'Sibsagar', 'Bhabua', 'Purba Champaran', 'Kawardha', 'Koriya', 'Raj Nandgaon', 'Daman', 'Delhi', 'Dahod', 'The Dangs', 'Sonepat', 'Yamuna Nagar', 'Anantnag (Kashmir South)', 'Bagdam', 'Baramula (Kashmir North)', 'Kupwara (Muzaffarabad)', 'Ladakh (Leh)', 'Punch', 'Hazaribag', 'Pashchim Singhbhum', 'Purba Singhbhum', 'Sahibganj', 'Bangalore Urban', 'Chamrajnagar', 'Davanagere', 'Uttar Kannand', 'Pattanamtitta', 'Kavaratti', 'East Nimar', 'West Nimar', 'Bid', 'Buldana', 'Garhchiroli', 'Gondiya', 'Greater Bombay', 'East Imphal', 'West Imphal', 'Jaintia Hills', 'Ri-Bhoi', 'Angul', 'Baragarh', 'Bolangir', 'Jagatsinghpur', 'Jajpur', 'Keonjhar', 'Puducherry', 'Firozpur', 'Nawan Shehar', 'Chittaurgarh', 'Dhaulpur', 'Jalor', 'Jhunjhunun', 'East', 'North Sikkim', 'South Sikkim', 'West Sikkim', 'Chennai', 'Kancheepuram', 'Nilgiris', 'Thoothukudi', 'Tiruchchirappal

### Use some language processing to find similarities of the indices of the dataframes that should be merged
Now find the mapping between centroid_keys_not_in_endDfKeys and endDfKeys_not_in_centroidKeys using string similarity:

In [130]:
import difflib as d

def seqmak(str1,str2):
    return d.SequenceMatcher(None, str1, str2).ratio()
sm=seqmak
mapping_notfound = []
for centroid_key in centroid_keys_not_in_endDfKeys:
    mapping_notfound.append({centroid_key : np.array([[x2, sm(centroid_key.upper(),x2)] for x2 in endDfKeys_not_in_centroidKeys])})
#mapping_notfound

[{'Andaman Islands': array([['24 PARAGANAS NORTH', '0.24242424242424243'],
         ['24 PARAGANAS SOUTH', '0.30303030303030304'],
         ['AGAR MALWA', '0.48'],
         ['ALIRAJPUR', '0.16666666666666666'],
         ['AMETHI', '0.2857142857142857'],
         ['AMROHA', '0.2857142857142857'],
         ['ANANTNAG', '0.43478260869565216'],
         ['ANJAW', '0.3'],
         ['ANUGUL', '0.2857142857142857'],
         ['ARWAL', '0.3'],
         ['BADGAM', '0.38095238095238093'],
         ['BAKSA', '0.2'],
         ['BALANGIR', '0.34782608695652173'],
         ['BALOD', '0.2'],
         ['BALODA BAZAR', '0.37037037037037035'],
         ['BANDIPORA', '0.3333333333333333'],
         ['BARABANKI', '0.25'],
         ['BARAMULLA', '0.4166666666666667'],
         ['BARDHAMAN', '0.5'],
         ['BARGARH', '0.18181818181818182'],
         ['BARNALA', '0.45454545454545453'],
         ['BEED', '0.10526315789473684'],
         ['BEMETARA', '0.17391304347826086'],
         ['BENGALURU URBAN', '0.1

Now for each centroid_key extract the endDfKey that has the highest similarity

In [131]:
count = 0
threshhold=0.6
most_similar = {}
excluded = {}
for key,dic in zip(centroid_keys_not_in_endDfKeys, mapping_notfound):
    mmax =  mapping_notfound[count][key][:,1].astype(float).max()
    if mmax > threshhold:
        max_index = mapping_notfound[count][key][:,1].astype(float).argmax()
        most_similar[key]= (mapping_notfound[count][key][max_index,0], mmax)
    else:
        max_index = mapping_notfound[count][key][:,1].astype(float).argmax()
        excluded[key] = (mapping_notfound[count][key][max_index,0], mmax)
    count +=1

In [132]:
##############################
#most_similar
##############################

{'Nicobar Islands': ('NICOBARS', 0.6956521739130435),
 'Nellore': ('SPSR NELLORE', 0.7368421052631579),
 'Vishakhapatnam': ('VISAKHAPATANAM', 0.9285714285714286),
 'Upper Dibang Valley': ('DIBANG VALLEY', 0.8125),
 'Dhuburi': ('DHUBRI', 0.9230769230769231),
 'North Cachar Hills': ('NORTH GARO HILLS', 0.8235294117647058),
 'Sibsagar': ('SIVASAGAR', 0.8235294117647058),
 'Purba Champaran': ('PURBI CHAMPARAN', 0.9333333333333333),
 'Kawardha': ('KABIRDHAM', 0.7058823529411765),
 'Koriya': ('KOREA', 0.7272727272727273),
 'Raj Nandgaon': ('RAJNANDGAON', 0.9565217391304348),
 'Daman': ('RAMBAN', 0.7272727272727273),
 'The Dangs': ('THE NILGIRIS', 0.6666666666666666),
 'Sonepat': ('SONIPAT', 0.8571428571428571),
 'Yamuna Nagar': ('YAMUNANAGAR', 0.9565217391304348),
 'Bagdam': ('BADGAM', 0.8333333333333334),
 'Punch': ('POONCH', 0.7272727272727273),
 'Hazaribag': ('HAZARIBAGH', 0.9473684210526315),
 'Pashchim Singhbhum': ('EAST SINGHBUM', 0.7096774193548387),
 'Purba Singhbhum': ('EAST SINGHBU

In [133]:
#tochange: 'South 24 Parganas':'24 PARAGANAS SOUTH'
#todelete = ['West Midnapore','Uttar Dinajpur','Hugli','Lakhimpur Kheri','East','East Imphal','West Imphal','Greater Bombay','Baramula (Kashmir North)','Dahod','Delhi','West Sikkim','South Sikkim','North Sikkim']
#what about 'North Cachar Hills'?
todelete = ['West Sikkim','South Sikkim','North Sikkim']

most_similar['South 24 Parganas'] = ('24 PARAGANAS SOUTH',np.nan)
print(most_similar['North 24 Parganas'])
for todel in todelete:
    excluded[todel] = most_similar[todel]
    most_similar.pop(todel)
print(most_similar)
#'Pashchim Singhbhum', 'Purba Singhbhum'



('24 PARAGANAS NORTH', 0.6285714285714286)
{'Nicobar Islands': ('NICOBARS', 0.6956521739130435), 'Nellore': ('SPSR NELLORE', 0.7368421052631579), 'Vishakhapatnam': ('VISAKHAPATANAM', 0.9285714285714286), 'Upper Dibang Valley': ('DIBANG VALLEY', 0.8125), 'Dhuburi': ('DHUBRI', 0.9230769230769231), 'North Cachar Hills': ('NORTH GARO HILLS', 0.8235294117647058), 'Sibsagar': ('SIVASAGAR', 0.8235294117647058), 'Purba Champaran': ('PURBI CHAMPARAN', 0.9333333333333333), 'Kawardha': ('KABIRDHAM', 0.7058823529411765), 'Koriya': ('KOREA', 0.7272727272727273), 'Raj Nandgaon': ('RAJNANDGAON', 0.9565217391304348), 'Daman': ('RAMBAN', 0.7272727272727273), 'The Dangs': ('THE NILGIRIS', 0.6666666666666666), 'Sonepat': ('SONIPAT', 0.8571428571428571), 'Yamuna Nagar': ('YAMUNANAGAR', 0.9565217391304348), 'Bagdam': ('BADGAM', 0.8333333333333334), 'Punch': ('POONCH', 0.7272727272727273), 'Hazaribag': ('HAZARIBAGH', 0.9473684210526315), 'Pashchim Singhbhum': ('EAST SINGHBUM', 0.7096774193548387), 'Purba Si

In [134]:
excluded

{'Andaman Islands': ('SOUTH ANDAMANS', 0.5517241379310345),
 'Cuddapah': ('KADAPA', 0.5714285714285714),
 'Bhabua': ('BULDHANA', 0.5714285714285714),
 'Delhi': ('AMETHI', 0.5454545454545454),
 'Dahod': ('BALOD', 0.6),
 'Anantnag (Kashmir South)': ('ANANTNAG', 0.5),
 'Baramula (Kashmir North)': ('24 PARAGANAS NORTH', 0.5714285714285714),
 'Kupwara (Muzaffarabad)': ('KAIMUR (BHABUA)', 0.4864864864864865),
 'Ladakh (Leh)': ('LEH LADAKH', 0.5454545454545454),
 'Kavaratti': ('KADAPA', 0.5333333333333333),
 'Bid': ('BEED', 0.5714285714285714),
 'Greater Bombay': ('GANDERBAL', 0.5217391304347826),
 'East Imphal': ('EAST SINGHBUM', 0.5833333333333334),
 'West Imphal': ('WEST SINGHBHUM', 0.56),
 'Chennai': ('KHERI', 0.5),
 'Thoothukudi': ('HOOGHLY', 0.4444444444444444),
 'Jyotiba Phule Nagar': ('KANPUR NAGAR', 0.5806451612903226),
 'Lakhimpur Kheri': ('DINAJPUR DAKSHIN', 0.5161290322580645),
 'Dakshin Dinajpur': ('KUSHI NAGAR', 0.5925925925925926),
 'East Midnapore': ('EAST DISTRICT', 0.5185185

In [135]:
flipped={}
for key, value in final_map_centroid_to_endDf.items():
    if value not in flipped:
        flipped[value] = [key]
    else:
        flipped[value].append(key)
  
# printing result
#print("final_dictionary", str(flipped))
for val in flipped.values():
    if len(val)>1:
        print(val)



After inspecting the code these had to removed manually because else there would be conflicts

In [136]:
#remove doubles
excluded['Pashchim Singhbhum'] = most_similar['Pashchim Singhbhum']
excluded['Purba Singhbhum'] = most_similar['Purba Singhbhum']
most_similar.pop('Pashchim Singhbhum')
most_similar.pop('Purba Singhbhum')
excluded['The Dangs'] = most_similar['The Dangs']
most_similar.pop('The Dangs')

('THE NILGIRIS', 0.6666666666666666)

In [137]:
most_similar['The Dangs']

KeyError: 'The Dangs'

In [138]:
#this is what we will add to the match_centroidtoendDf

print(len(match_centroidtoendDf)+len(most_similar))
cleaned_most_similar = {}
for key in most_similar.keys():
    cleaned_most_similar[key] = most_similar[key][0]

#now join match_centroidtoendDf with cleaned_most_similar
final_map_centroid_to_endDf = {**match_centroidtoendDf,**cleaned_most_similar}
len(final_map_centroid_to_endDf)

561


561

In [139]:
len(most_similar)

67

In [140]:
#end_df = end_df.set_index("dist")
todelete_from_endDf = list(set(end_df["dist"])-set(final_map_centroid_to_endDf.values()))
todelete_from_centroid_dist = list(set(centroid_dist.index)-set(final_map_centroid_to_endDf.keys()))
todelete_from_endDf2 = set(final_map_centroid_to_endDf)-set(end_df.index)
todelete_from_endDf2 = set(end_df["dist"])-set(final_map_centroid_to_endDf.values())

KeyError: 'dist'

In [141]:
len(todelete_from_centroid_dist)

31

In [142]:
#end_df = end_df.set_index("dist")
#final_end_df = end_df.drop(index=todelete_from_endDf)
final_centroid_dist = centroid_dist.drop(index=todelete_from_centroid_dist)

In [143]:
end_df_eff = end_df_eff.set_index("dist")
final_end_df_eff = end_df_eff.drop(index=todelete_from_endDf)

KeyError: "None of ['dist'] are in the columns"

In [144]:
len(final_centroid_dist)

558

In [145]:
print(len(final_map_centroid_to_endDf))
print(len(final_end_df))
print(len(final_centroid_dist))

561
558
558


In [146]:
adapted_from_centroids = {final_map_centroid_to_endDf[x] for x in final_centroid_dist.index}
print(len(final_map_centroid_to_endDf))
print(len(final_centroid_dist.index))

561
558


In [147]:
set(final_end_df.index)-set(final_map_centroid_to_endDf.values())

set()

In [148]:
len(set(final_end_df.index))
len(set(final_map_centroid_to_endDf.keys()))

561

In [149]:
final_end_df_eff.head()

,crop,eff
dist,,
24 PARAGANAS NORTH,Coconut,3258.620190
24 PARAGANAS SOUTH,Coconut,2244.891828
ADILABAD,Sugarcane,82.385214
AGRA,Sugarcane,48.458864
AHMADABAD,Sugarcane,67.777778


In [162]:
final_centroid_dist.head()

,State,Latitude,Longitude
District,,,
Nicobar Islands,Andaman and Nicobar,7.835291,93.511601
Adilabad,Andhra Pradesh,19.284514,78.813212
Anantapur,Andhra Pradesh,14.312066,77.460158
Chittoor,Andhra Pradesh,13.331093,78.927639
East Godavari,Andhra Pradesh,16.782718,82.243207


Rename indices in the final_centroid_dist according to final_map_centroid_to_endDf and then unite the dataframes

In [161]:
renamed_centroids = final_centroid_dist.rename(index=final_map_centroid_to_endDf)
renamed_centroids.head()

,State,Latitude,Longitude
District,,,
NICOBARS,Andaman and Nicobar,7.835291,93.511601
ADILABAD,Andhra Pradesh,19.284514,78.813212
ANANTAPUR,Andhra Pradesh,14.312066,77.460158
CHITTOOR,Andhra Pradesh,13.331093,78.927639
EAST GODAVARI,Andhra Pradesh,16.782718,82.243207


Now merge the dataframes!!!

In [165]:
final_dataframe_eff = pd.concat([renamed_centroids,final_end_df_eff], axis=1)
final_dataframe_eff = final_dataframe_eff.reset_index()
#final_dataframe["District"] = final_dataframe.index
final_dataframe_eff =final_dataframe_eff.rename(columns = {"index":"District", "Efficacy(production(tons) per area(ha))": "Efficacy (APMC unit for crop per area (ha))", "crop":"Crop"})
final_dataframe_eff.head()

,District,State,Latitude,Longitude,Crop,eff
0,NICOBARS,Andaman and Nicobar,7.835291,93.511601,Coconut,3734.411083
1,ADILABAD,Andhra Pradesh,19.284514,78.813212,Sugarcane,82.385214
2,ANANTAPUR,Andhra Pradesh,14.312066,77.460158,Coconut,6257.169944
3,CHITTOOR,Andhra Pradesh,13.331093,78.927639,Coconut,6263.421153
4,EAST GODAVARI,Andhra Pradesh,16.782718,82.243207,Coconut,12826.975634


Now save the final dataframe!!!!! This now is a dataframe with the crop that has the corresponding highest efficacy (see above)

In [167]:
final_dataframe_eff.to_csv("data_india/dist_lat_long_crop_eff.csv")